In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch import exceptions as es_exceptions
from langchain.embeddings import SentenceTransformerEmbeddings
import pandas as pd
import os

/Users/leozangulo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
try:
    esuser = "<USERNAME>"
except KeyError:
    esuser = input("Please enter your Elasticsearch user name (hit enter): ")
try:
    espassword = "<PASSWORD>"
except KeyError:
    espassword = getpass.getpass("Please enter your Elasticsearch password (hit enter): ")
try:
    eshost = "<ELASTIC_URL>"
except KeyError:
    eshost = input("Please enter your Elasticsearch hostname (hit enter): ")
try:
    esport = "XXXX"
except KeyError:
    esport = input("Please enter your Elasticsearch port number (hit enter): ")


In [3]:
es_ssl_fingerprint = !openssl s_client -connect $eshost:$esport  -showcerts </dev/null 2>/dev/null | openssl x509 -fingerprint -sha256 -noout -in /dev/stdin
es_ssl_fingerprint = es_ssl_fingerprint[0].split("=")[1]
es_ssl_fingerprint

'22:B8:75:5A:05:3F:6D:7D:D7:43:CB:07:63:41:0B:5B:B0:AC:2F:C9:9F:BF:8C:CE:3C:D4:42:5F:B0:92:E4:A9'

In [4]:
elastic_client = Elasticsearch([f"https://{esuser}:{espassword}@{eshost}:{esport}"],
                              basic_auth=(esuser, espassword),
                              request_timeout=None,
                              ssl_assert_fingerprint=es_ssl_fingerprint)

In [6]:
# Verificar que la conexión este creada
try:
    info = elastic_client.info()
    print("Conexión exitosa. Info del clúster:")
    print(info)
except es_exceptions.AuthenticationException as e:
    print("Error de autenticación:", e.info)
except es_exceptions.ConnectionError as e:
    print("Error de conexión:", e.info)
except es_exceptions.AuthorizationException as e:
    print("Error de autorización:", e.info)
except es_exceptions.TransportError as e:
    print("Error general de transporte:", e.info)
except Exception as e:
    print("Otro error:", str(e))

Conexión exitosa. Info del clúster:
{'name': 'm-1.63ae1f09-63d0-42c1-a834-11658d84d119.b3580ea2cd7943f795db3d4b2f063fc7.c5kmhkid0ujpmrucb800.databases.appdomain.cloud', 'cluster_name': '63ae1f09-63d0-42c1-a834-11658d84d119', 'cluster_uuid': '1OewP8bXSO2Y48FwiRaqeg', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Selección del modelo

In [7]:
emb_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
dims = emb_func.client.get_sentence_embedding_dimension()

/var/folders/h_/w76hwqrd6kv3q8ps8cm4_y980000gn/T/ipykernel_87367/2620494856.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/leozangulo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Creación del índice y del esquema
index_name = "test_nds_v2"

mapping = {
        "properties": {
                "text": {
                        "type": "text"
                    },
                "embedding": {
                        "type": "dense_vector",
                        "dims": dims,
                        "index": True,
                        "similarity": "l2_norm"
                    }
            }
    }
if elastic_client.indices.exists(index=index_name):
    elastic_client.indices.delete(index=index_name)
    
elastic_client.indices.create(index=index_name, mappings=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_nds_v2'})

## Definición de documentos

In [9]:
# Seleccionar todos los .txt
scl = []
for file in os.listdir("./"):
    if file.endswith(".txt"):
        print(os.path.join(file))
        scl.append(file)

In [45]:
documents_filename = "./qna.csv"
documents = pd.read_csv(documents_filename, header=0)
c = documents['Compositor']
p = documents['Pregunta']
r = documents['Respuesta']
documents['resumen'] = 'Tema: '+c+', Pregunta: '+p+', Respuesta: '+r


cols = documents.columns.tolist()
cols = ['resumen'] + [col for col in cols if col != 'resumen']
documents = documents[cols]

documents[:10]

,resumen,Compositor,Pregunta,Respuesta
0,"Tema: Mozart, Pregunta: ¿Dónde nació Mozart?, ...",Mozart,¿Dónde nació Mozart?,"Mozart nació en Salzburgo, que en su época era..."
1,"Tema: Mozart, Pregunta: ¿A qué edad empezó a c...",Mozart,¿A qué edad empezó a componer?,"Comenzó a componer desde los 5 años, escribien..."
2,"Tema: Mozart, Pregunta: ¿Cuál es su nombre com...",Mozart,¿Cuál es su nombre completo?,Su nombre completo era Johannes Chrysostomus W...
3,"Tema: Mozart, Pregunta: ¿Qué significa “Amadeu...",Mozart,¿Qué significa “Amadeus”?,"Significa 'amado de Dios' en latín, aunque tam..."
4,"Tema: Mozart, Pregunta: ¿Qué famosa ópera comp...",Mozart,¿Qué famosa ópera compuso en 1786?,Compuso 'Las bodas de Fígaro' (Le nozze di Fig...
5,"Tema: Mozart, Pregunta: ¿Cuál es su ópera cons...",Mozart,¿Cuál es su ópera considerada la más oscura y ...,"Sin duda 'Don Giovanni', que mezcla comedia y ..."
6,"Tema: Mozart, Pregunta: ¿Qué obra dejó inconcl...",Mozart,¿Qué obra dejó inconclusa al morir?,"Su famoso Réquiem en Re menor, que estaba comp..."
7,"Tema: Mozart, Pregunta: ¿Quién terminó su Réqu...",Mozart,¿Quién terminó su Réquiem?,"Fue su alumno Franz Xaver Süssmayr, quien comp..."
8,"Tema: Mozart, Pregunta: ¿En qué ciudad murió M...",Mozart,¿En qué ciudad murió Mozart?,"Murió en Viena, la capital de Austria, donde p..."
9,"Tema: Mozart, Pregunta: ¿A qué edad murió?, Re...",Mozart,¿A qué edad murió?,"Murió muy joven, a los 35 años, dejando más de..."


In [46]:
texts = documents.resumen.tolist()
embedded_docs = emb_func.embed_documents(texts)

In [58]:
# Guardar los documentos en la DB
for i, (text, vector) in enumerate(zip(texts, embedded_docs)):
    document_l = {"embedding": vector, 'text': text}
    elastic_client.index(index=index_name, document=document_l)
    

elastic_client.indices.refresh(index=index_name)

test_nds_v2


ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

In [60]:
# Mostrar cantidad de indices
try:
    response = elastic_client.count(index="test_nds_v2")
    total_docs = response['count']
    print(f"📊 El índice '{indice}' tiene {total_docs} documentos.")
except Exception as e:
    print("❌ Error al obtener el conteo:", e)

📊 El índice '.ent-search-actastic-oauth_access_tokens_v2' tiene 50 documentos.


In [ ]:
#document_list

In [54]:
try:
    response = elastic_client.count(index="test_nds_v2")
    total_docs = response['count']
    print(f"📊 El índice '{indice}' tiene {total_docs} documentos.")
except Exception as e:
    print("❌ Error al obtener el conteo:", e)

📊 El índice '.ent-search-actastic-oauth_access_tokens_v2' tiene 0 documentos.


In [63]:
# Ver cuantos indices hay
indices = elastic_client.indices.get_alias(index="*").keys()
for indice in indices:
    print("-", indice)

- .ent-search-actastic-workplace_search_accounts_v16
- .ent-search-actastic-workplace_search_search_groups_v4-name-unique-constraint
- .ent-search-actastic-crawler2_robots_txts
- .ent-search-actastic-workplace_search_pre_content_sources_v3
- .ent-search-actastic-crawler_crawl_requests_v7
- .ent-search-esqueues-me_queue_v1_process_crawl2
- .ent-search-actastic-reindex_jobs_v3
- .ent-search-actastic-workplace_search_role_mappings_v8
- .kibana_8.15.0_001
- .ent-search-actastic-search_relevance_suggestion_update_process_v1
- .apm-custom-link
- .ent-search-actastic-connectors_jobs_v5
- .ml-annotations-000001
- .ent-search-actastic-workplace_search_content_sources_v23
- .internal.alerts-observability.uptime.alerts-default-000001
- .ent-search-actastic-users_v7-auth_source-elasticsearch_username-unique-constraint
- .ent-search-actastic-crawler_process_crawls
- .apm-source-map
- .ent-search-actastic-users_v7-email-unique-constraint
- .ent-search-actastic-crawler2_configurations_v2-index_name-u